<a href="https://colab.research.google.com/github/giustinod/Fine-Tuning-Llama-2LLM/blob/main/Fine_tune_Llama_3_2_3B_Instruct_with_Unsloth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

check also: https://medium.com/@amrstech/fine-tuning-made-easy-with-unsloth-and-colab-e0993f3f4c07

In [3]:
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

from huggingface_hub import login
from google.colab import userdata
import wandb

import torch
from trl import SFTConfig, SFTTrainer, DataCollatorForCompletionOnlyLM
from datasets import load_dataset
from transformers import TrainingArguments, TextStreamer
from unsloth.chat_templates import get_chat_template
from unsloth import FastLanguageModel, is_bfloat16_supported

hf_token = userdata.get('HF_TOKEN')
login(hf_token)

wb_token = userdata.get("wandb")

wandb.login(key=wb_token)
run = wandb.init(
    project='Fine-tune Mathstral-7B-v0.1 on AZService', #
    job_type="training",
    anonymous="allow"
)

# caricamento modello e test
max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    # deepseek-ai/deepseek-math-7b-instruct, unsloth/Llama-3.2-3B-Instruct
    model_name = "mistralai/Mathstral-7B-v0.1",
    # max_seq_length = max_seq_length,
    # dtype = dtype,
    # load_in_4bit = load_in_4bit,
    token = hf_token,
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

alpaca_prompt = """The task is to pair requirements and tests for an object. Write a response that appropriately completes the request for the input.

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    inputs       = examples["premises"]
    outputs      = examples["conclusion"]
    texts = []
    for input, output in zip(inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

dataset = load_dataset("azservice/specs_and_variables", split="train", token = hf_token)
dataset = dataset.map(formatting_prompts_func, batched = True,)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    # dataset_text_field = "text",
    # max_seq_length = max_seq_length,
    # dataset_num_proc = 2,
    # packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        # neftune_noise_alpha = 5,
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        # warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        # max_steps = 1,
        # learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        # logging_steps = 1,
        # optim = "adamw_8bit",
        # weight_decay = 0.01,
        # lr_scheduler_type = "linear",
        # seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

import os
os.environ['UNSLOTH_RETURN_LOGITS'] = '1'

trainer.train()

model = FastLanguageModel.for_inference(model)

messages = [
    {"from": "human", "value": "What about design specification for route protection in the rail network?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
).to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(input_ids=inputs, streamer=text_streamer, max_new_tokens=128, use_cache=True)

# Save locally as sharded model files
model.save_pretrained_merged("TestLogica", tokenizer, save_method = "merged_16bit",)
# Push to Huggingface hub - replace the space username as required
model.push_to_hub_merged("azservice/TestLogica-Mathstral-7B-v0.1", tokenizer, save_method = "merged_16bit", token = hf_token)


Found existing installation: unsloth 2025.3.19
Uninstalling unsloth-2025.3.19:
  Successfully uninstalled unsloth-2025.3.19
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-req-build-6uthw3wd
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-req-build-6uthw3wd
  Resolved https://github.com/unslothai/unsloth.git to commit f6f5e6397acfb5737dcb2062a72e860cc3657f6f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2025.3.19-py3-none-any.whl size=192490 sha256=47e84cdac3f1d3ca4cb1e766ec88f1a3d84b3bfffd0df1dd9387ce1c5b16362c
  Stored in directory: /tmp/pip-ephem-wheel-cache-0blwv2fl/wheels/d1/17/05/850ab10c33284a4763b0595cd8ea9d01fce6e221cac24b3c01
Successfully built unsloth


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


==((====))==  Unsloth 2025.3.19: Fast Mistral patching. Transformers: 4.51.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

mistralai/Mathstral-7B-v0.1 does not have a padding token! Will use pad_token = [control_748].


...

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/4201 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 4,201 | Num Epochs = 3 | Total steps = 1,575
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/7,000,000,000 (0.60% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss


KeyboardInterrupt: 

Il tentavivo è di usare un modello specifico per operazioni metematiche

cfr. https://medium.com/@vinyes.marina/large-language-models-and-math-a-review-of-approaches-and-progress-b58c76e7716e

cfr. https://medium.com/@marvashahid09/fine-tuning-deepseekmath-for-solving-math-problems-10997bc095d4

In [ ]:
# !pip install transformers==4.45.0 accelerate==0.26.0 bitsandbytes>=0.43.3
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

import torch
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.version.cuda)
!pip show bitsandbytes
import bitsandbytes
print(bitsandbytes.__version__)
import bitsandbytes as bnb
x = torch.randn(10, device="cuda")
y = bnb.functional.quantize_4bit(x)
print("Quantization worked!")
import bitsandbytes.nn
import bitsandbytes.functional
print("Submodules imported successfully!")

import transformers
transformers.utils.is_bitsandbytes_available = lambda: True
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import os
import gc

from peft import LoraConfig, get_peft_model

torch.cuda.empty_cache()
gc.collect()

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

# Define model and tokenizer
model_name = "deepseek-ai/deepseek-math-7b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Set padding token if not already set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

# Define LoRA configuration
lora_config = LoraConfig(
    r=16,  # Rank of the LoRA adaptation
    lora_alpha=32,  # Scaling factor
    target_modules=["q_proj", "v_proj"],  # Target attention layers (adjust based on model architecture)
    lora_dropout=0.05,  # Dropout for regularization
    bias="none",  # No bias in LoRA layers
    task_type="CAUSAL_LM",  # Task type for causal language modeling
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()  # Verify trainable parameters

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.7/192.7 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Found existing installation: unsloth 2025.3.19
Uninstalling unsloth-2025.3.19:
  Successfully uninstalled unsloth-2025.3.19
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-req-build-8t8kxh44
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-req-build-8t8kxh44
  Resolved https://github.com/unslothai/unsloth.git to commit f6f5e6397acfb5737dcb2062a72e860cc3657f6f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2025.3.19-py3-none-any.whl size=192490 sha256=5c9fef4a9be24dcf6237640cbdd4659be15f5c382e1b4e228083940c6aaaa476
  Stored in directory: /tmp/pip-ephem-wheel-cache-pwmpr1qf/wheels/d1/17/05/850ab10c33284a4763b0595cd8ea9d01fce6e221cac24b3c01
Successfully built unsloth
2.6.0+cu124
True
12.4
Name: bitsandbytes
Version: 0.45.5
Summary: k-bit optimizers and matrix multiplication routines.
Hom

tokenizer_config.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.61M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/594 [00:00<?, ?B/s]

...

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.97G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.6k [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

trainable params: 7,864,320 || all params: 6,918,230,016 || trainable%: 0.1137


Al momento non ce la fa con la memoria

In [ ]:
from huggingface_hub import login
from google.colab import userdata
import wandb
import os
import torch
from trl import SFTConfig, SFTTrainer, DataCollatorForCompletionOnlyLM
from datasets import load_dataset
from transformers import TrainingArguments, TextStreamer

hf_token = userdata.get('HF_TOKEN')
login(hf_token)

alpaca_prompt = """The task is to pair requirements and tests for an object. Write a response that appropriately completes the request for the input.

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    inputs       = examples["premises"]
    outputs      = examples["conclusion"]
    texts = []
    for input, output in zip(inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

dataset = load_dataset("azservice/specs_and_variables", split="train", token = hf_token)
dataset = dataset.map(formatting_prompts_func, batched = True,)

# Define training arguments
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    args = TrainingArguments(
        neftune_noise_alpha = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        logging_steps = 1,
        optim = "adamw_8bit",
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)
"""
training_args = TrainingArguments(
    output_dir="outputs",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,  # Adjust based on GPU memory (T4x2)
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,  # Effective batch size = 2 * 4 = 8
    # evaluation_strategy="epoch",
    # save_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    fp16=True,  # Use mixed precision for T4 GPU
    logging_dir="logs",
    logging_steps=10,
    # load_best_model_at_end=True,
    metric_for_best_model="loss",
    report_to="none",  # Disable wandb in Kaggle
    push_to_hub=False,
)

# Initialize Trainer
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    # eval_dataset=eval_dataset,
    # ndata_collator=data_collator,
    # compute_metrics=compute_metrics  # Uncomment if you want accuracy metrics
)
"""
# Train the model
trainer.train()

<ipython-input-3-8df36f6730dd>:37: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


OutOfMemoryError: CUDA out of memory. Tried to allocate 86.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 6.12 MiB is free. Process 2574 has 14.73 GiB memory in use. Of the allocated memory 13.82 GiB is allocated by PyTorch, and 793.31 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

Da: https://huggingface.co/spaces/Thziin/Tutorial/blob/5ace8366196059399d4387bb3d3b969007e5dfdc/app.py

Con il modello originario (unsloth/Meta-Llama-3.1-8B-bnb-4bit) è molto lento ma risponde, dopo il fine tuning restituisce:
Error during inference: 504 Server Error: Gateway Timeout for url: https://api-inference.huggingface.co/models/azservice/TestLogica-Pdfs/v1/chat/completions (Request ID: 0Tusix)
Model azservice/TestLogica-Pdfs time out

In [ ]:
%pip install gradio

import gradio as gr
from huggingface_hub import InferenceClient

# Safely initialize the inference client
def initialize_client():
    try:
        print("Initializing inference client...")
        client = InferenceClient("azservice/TestLogica-deepseek-math-7b-instruct")
        print("Inference client initialized successfully!")
        return client
    except Exception as e:
        print(f"Error initializing inference client: {e}")
        return None

client = initialize_client()

# Chatbot response logic
def respond(
    message,
    history: list[tuple[str, str]],
    system_message,
    max_tokens,
    temperature,
    top_p,
):
    if not client:
        return "Error: Inference client not initialized."

    messages = [{"role": "system", "content": system_message}]

    # Add historical interactions
    for val in history:
        if val[0]:
            messages.append({"role": "user", "content": val[0]})
        if val[1]:
            messages.append({"role": "assistant", "content": val[1]})

    # Add user message
    messages.append({"role": "user", "content": message})

    try:
        print("Sending request to model...")
        response = client.chat_completion(
            messages,
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
        ).choices[0].message.content
        print("Response received successfully!")
        return response
    except Exception as e:
        print(f"Error during inference: {e}")
        return f"Error during inference: {e}"

# Gradio interface
def launch_demo():
    try:
        demo = gr.ChatInterface(
            respond,
            additional_inputs=[
                gr.Textbox(value="You are a friendly Chatbot. Your name is Juninho.", label="System message"),
                gr.Slider(minimum=1, maximum=2048, value=512, step=1, label="Max new tokens"),
                gr.Slider(minimum=0.1, maximum=4.0, value=0.7, step=0.1, label="Temperature"),
                gr.Slider(
                    minimum=0.1,
                    maximum=1.0,
                    value=0.95,
                    step=0.05,
                    label="Top-p (nucleus sampling)",
                ),
            ],
        )
        demo.launch()
    except Exception as e:
        print(f"Error launching Gradio app: {e}")

if __name__ == "__main__":
    launch_demo()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 115.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 5.4 MB/s eta 0:00:00
Initializing inference client...
Inference client initialized successfully!


/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:338: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1be6a72e3402e74c7a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
